## <font color=blue> Lec2 UDP --- Client</font>   
-----------------------------------------------------------

#### UDP Ex2_1 Client

In [15]:
import socket
import datetime

In [16]:
MAX_BYTES = 65535
port = 5151

In [17]:
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

In [22]:
text = 'The time is {}'.format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
data = text.encode('ascii')
sock.sendto(data, ('127.0.0.1', port))

31

In [23]:
print('The OS assigned me the address {}'.format(sock.getsockname()))

The OS assigned me the address ('0.0.0.0', 52413)


In [24]:
data, address = sock.recvfrom(MAX_BYTES)  # Danger! See Chapter 2

In [25]:
text = data.decode('ascii')
print('The server {} replied {!r}'.format(address, text))

The server ('127.0.0.1', 5151) replied 'Your data was 31 bytes long'


#### UDP Ex2_2 Client

In [1]:
import argparse, random, socket, sys

In [2]:
MAX_BYTES = 65535
hostname = '127.0.0.1'
port = 5151

In [3]:
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.connect((hostname, port))

In [4]:
print('Client socket name is {}'.format(sock.getsockname()))

Client socket name is ('127.0.0.1', 59556)


In [12]:
text = 'This is another message'
data = text.encode('ascii')

In [16]:
delay = 0.1  # seconds
while True:
    sock.send(data)
    print('Waiting up to {} seconds for a reply'.format(delay))
    sock.settimeout(delay)
    try:
        data = sock.recv(MAX_BYTES)
    except socket.timeout as exc:
        delay *= 2  # wait even longer for the next request
        if delay > 2.0:
            raise RuntimeError('I think the server is down') from exc
    else:
        break   # we are done, and can stop looping
print('The server says {!r}'.format(data.decode('ascii')))

Waiting up to 0.1 seconds for a reply
Waiting up to 0.2 seconds for a reply
Waiting up to 0.4 seconds for a reply


ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。

#### UDP Ex2_3 Client

In [65]:
import argparse, random, socket, sys

In [66]:
zen = ''' \
The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!
'''

In [67]:
lines = zen.split('\n')

In [68]:
MAX_BYTES = 65535
hostname = '127.0.0.1'
port = 5151

In [69]:
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.connect((hostname, port))

In [70]:
print('Client socket name is {}'.format(sock.getsockname()))

Client socket name is ('127.0.0.1', 56406)


In [75]:
def send_reliable_data(sock, text, data_id = 0):
    str_id = "{:08d}".format(data_id)
    try_time = 0
    delay = 0.1  # seconds
    while try_time < 10:
        try_time = try_time + 1
        sock.send((str_id + text).encode('ascii'))
        print('Waiting up to {} seconds for a reply'.format(delay))
        sock.settimeout(delay)
        try:
            data = sock.recv(MAX_BYTES)
            if (len(data) > 8):
                if (data[0:8].decode('ascii') == str_id):
                    print('The server says {!r}'.format(data.decode('ascii')))
                    return True
                else:
                    print('data id is mismatch, data:{!r}\n'.format(data.decode('ascii')))
        except socket.timeout as exc:
            delay *= 2  # wait even longer for the next request
            if delay > 2.0:
                raise RuntimeError('I think the server is down') from exc
    return False

In [72]:
data_id = random.randint(100, 65000)

In [74]:
for line in lines:
    if not (send_reliable_data(sock, line, data_id)):
        break
    data_id = data_id + 1

Waiting up to 0.1 seconds for a reply
Waiting up to 0.2 seconds for a reply
Waiting up to 0.4 seconds for a reply
Waiting up to 0.8 seconds for a reply
Waiting up to 1.6 seconds for a reply


RuntimeError: I think the server is down